# Wichtige Funktionen zur Auswertung von Versuchen. 
Diese Vorlage kann kopiert werden und dann für den jeweiligen Versuch genutzt werden. 

Zunächst soll die Versuchsummer des jeweiligen Versuches angegeben werden, damit das automatische Speichern der Grafiken in das richtige Verzeichnis funktioniert.

Dieses Dokument ist an vielen Stellen (stark) an *Luca Hafner* orientiert.

## Versuchsnummer:

In [4]:
versuchsnummer = 000

## Import aller wichtigen Libaries

In [39]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.transforms as transforms
%matplotlib inline

import numpy as np
from numpy import exp, sqrt, log, pi

from scipy import odr
import scipy.optimize
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy.stats import poisson
from scipy.integrate import quad
from scipy.signal import find_peaks
from scipy.signal import argrelextrema, argrelmin, argrelmax
from scipy.special import factorial

import os
import pandas as pd
import pylab as py
from IPython.display import display, Math, Latex, HTML

import sympy as sp
from sympy import separatevars

## Definition der Funktionen

---
## Table of Contents:
* [Import von Dateien](#Import-von-Dateien)
* [Fehlerrechnung einer gegebenen Formel](#Fehlerrechnun-einer-gegebenen-Formel)
* [Automatisierter Latex-Export](#Automatisierter-Latex-Export)
    * [Export der Latex Formel](#Export-der-Latex-Formel)
    * [Ergebnisse plus Fehler](#Ergebnisse-plus-Fehler)
    * [Export als Tabelle](#Export-als-Tabelle)
---

##  Import von Dateien
Es sollen hier Funktionen definiert werden, die **CSV** und **TXT** -Dateien importieren. Diese sollen dann einfacher automatisiert in Latex-Tabellen eingefügt werden. 

In [94]:
"""
    Hier sollen Messwerte aus CSV und TXT
"""

def placeholder():
    return

## Fehlerrechnung einer gegebenen Formel

**Gauss Fehler Fortpflanzung (GFF)**:

Die *Methode gff* nimmt zwei Argumente, zum einen eine *sympy (sp)* Funktion, zum anderen einen Array aller fehlerbehafteter Größen.

In [ ]:
def gff(func, errPronePar):
    """
    Gaussian Error Propergation
    """
    # Variablen definieren
    error = 0
    errProneParamters = []
    for errPar in errPronePar:
        d = sp.symbols('d' + errPar.name)
        partial = sp.diff(func, errPar) * d  # Die Funktion wird nach der fehlerbehafteten Variable abgeleitet
        error = error + partial**2 # Fehler werden quadratisch aufsummiert
        errProneParamters.append((errPar,d))
    absolut_err=sp.simplify(sp.sqrt(error),rational = True)             
    relativ_err=sp.simplify(sp.sqrt(error/func**2),rational = True)
    return absolut_err, relativ_err, errProneParamters

Beispiel gff:

In [ ]:
# Drei Messgrößen x, y und z werden definiert.
x = sp.Symbol('x')
y = sp.Symbol('y')
z = sp.Symbol('z')

params = [x, y, z]

# Darstellen einer Funktion
f = x**2 * sp.sin(y) / sp.sqrt(z)


# Unsicherheiten (bspw. Ablesefehler)
dx_val = 0.05  
dy_val = 0.02 
dz_val = 0.10  


# Ausführen von gff
abs_err, rel_err, param_symbols = gff(f, params)

print("Absoluter Fehler (delta f):")
sp.pprint(abs_err)

print("\nRelativer Fehler (delta f / f):")
sp.pprint(rel_err)

## Automatisierter Latex-Export

Es wird eine dedizierte Datei für die Ergebnisse geben, in der die Ergebnisse als Latex-Variable stehen. Diese können dann im Latex-Code implementiert werden. Wurde eine Größe falsch bestimmt, so kann diese im Python-Code neu berechnet werden und wird automatisiert im neuen Dokument (nach einem *Recompile*) korrigiert dargestellt.

## Ersstellen einer dedizierten **TEX** -Datei

Diese beinhaltet:
* Formel, sowie Formel nach gff
* Berechnete Werte und deren Fehler
    * Wert + Fehler
    * Tabellen Export


### Export der Latex Formel

In [133]:
def function_to_latex(f):
    """
    Zeigt die Formel als gerenderte Math-Darstellung und darunter
    den Latex-Quelltext, der per Button kopiert werden kann. (Für leichtere Benutzung als HTML)
    """
    # print("Die gegebene Funktion lautet: \n")
    display(Math(sp.latex(f, long_frac_ratio=2).replace('d__', r'\Delta ')))

    # print("Hier ist der dazugehörige Latex code: \n")
    latex_str = sp.latex(f, long_frac_ratio=2).replace('d__', r'\Delta ')
    html = f"""
    <div style="margin-top:0.5em;">
        <code id="latex-code-{id(f)}">
            {latex_str}
        </code>
        <button onclick="
            const tex_as_txt = document.getElementById('latex-code-{id(f)}').innerText;
            navigator.clipboard.writeText(tex_as_txt)
        " style="
            margin-left:8px;
            padding:2px 6px;
            cursor:pointer;
        ">
            Kopieren
        </button>
    </div>
    """
    display(HTML(html))

Beispiel function_to_latex:

In [ ]:
function_to_latex(f)

### Ergebnisse plus Fehler

In [ ]:
def calc_with_err(func, errFunc, values):
    """
    Methode zum berechnen von Werten und deren Fehler.

    Parameter
    ----------
    func: sympy Funktion mit Parametern. 
    errFunc: die zu func gehöhrende sympy Fehler-Funktion nach gff
    values: Werte, die in die Funktionen eingesetzt werden.
        Als array von Tupeln der Form [(a,da),(b,db),...] oder als array/liste [a,da,b,db,...] 
        (Reihenfolge muss die sein, in der die Argumente in der Funktion genommen werden)
    """

    #Falls der Input in mehrere Tupel aufgeteilt ist, werden diese zu einem Array zusammengefügt 
    if (np.ndim(values) != 1):                    
        values = np.concatenate(values)
    #print(values)
    result = func(*values[::2])
    uncertainty = errFunc(*values)
    return result, uncertainty

In [200]:
def calc_everything(function, params, data, params_without_error=[]):
    """
    Returned Berechneten Wert plus Fehler. Zudem werden die Formeln als Latex-Code ausgeworfen.

    Parameter
    ----------
    **function**: sympy Funktion mit Parametern. In diese werden die Messwerte eingesetzt.

    **params**: Parameter der Funktion. Diese werden als Array von Sympy-Symbolen gebraucht. Bspw. [x, y, z]

    **data**: 2D-Array mit den Messdaten, sodass die Zeilen die Form haben: [Parameter 1, Fehler Parameter 1, Parameter 2,...]
        Die Funktion wird zeilenweise angewandt. Wird kein Fehler für einen Parameter angenommen, kann diese Spalte entweder mit dem Wert 0 
        an die Funktion gegeben werden oder ganz weggelassen werden. Dann muss allerdings der betreffende Parameter bei params_without_error angegeben werden.

    **params_without_error**: Alle Parameter zu denen kein Fehler explizit in den Daten angegeben ist. Dieser wird auf 0 gesetzt und kommt dann
        auch nicht in der Latex Form der Fehlerformel vor

    """
    # Expand data to include the uncertainty 0 for values with no assigned uncertainty
    # Hat zubeginn die Form: [[0. 0. 0. 0. 0. 0. ...]]. 
    # Bei jeder Iteration werden dann die Parameter und deren Fehler hinzugefügt: i=1 -> [[par1 errPar1 0. 0. 0. 0. ...]]
    exp_data = np.zeros((data.shape[0],data.shape[1]+len(params_without_error)))
    i = 0      # läuft durch die Parameter
    j = 0      # läuft durch die expanded data
    z = 0      # läuft durch die eingegebene data, also die Messwerte und deren Fehler
    # Läuft durch jeden Parameter und seinen Fehler
    while (i < len(params)):
        # Checkt, ob der Parameter Fehlerbehaftet ist. Wenn, dann wird an j-ter Stelle des exp_data der z-te Parameter aus data einegfügt.
        if (params[i] in params_without_error):
            exp_data[:,j] = data[:,z]
            i = i + 1
            j = j + 2
            z = z + 1
        else:
            exp_data[:,j] = data[:,z]
            exp_data[:,j+1] = data[:,z+1]
            i = i + 1
            j = j + 2
            z = z + 2
    # print(exp_data)

    # Create variable that stores parameters that have no assigned uncertainty    
    params_with_error = []
    j = 0
    for n in np.arange(0,len(params)):
        if not (params[n] in params_without_error):
            params_with_error.append(params[n])
            j = j + 1
    
    # Get the given function and error function as numpy functions
    f = sp.lambdify(params,function)
    absolut_err, relativ_err, parameters = gff(function,params) # Gauss Fehlerfortpflanzung
    err_abs = sp.lambdify(np.concatenate(parameters),absolut_err)

    # Calculate the results for each row of data
    results = np.zeros((data.shape[0],2))
    for n in np.arange(0,data.shape[0]):
        results[n,:] = calc_with_err(f, err_abs, exp_data[n,:])
        # print(calc_with_err(f, err_abs, exp_data[n]))
    
    if (len(results) < 10):
        print("Results:")
        print(results)

    # Substitutes 0 for the uncertainty of the parameters without error, so it doesnt show up in the Latex Code
    for p in params_without_error:
        absolut_err = absolut_err.subs('d'+p.name,0)
    for p in params_without_error:
        relativ_err = relativ_err.subs('d'+p.name,0)

    # Wiedergabe des Latex-Codes
    function = sp.simplify(function,symbols = params, rational= True)
    function = sp.separatevars(function)
    
    print("gegebene Funktion:")
    function_to_latex(function)

    print("Formel des absoluten Fehlers der gegebenen Funktion:")
    function_to_latex(absolut_err)

    print("Formel des relativen Fehlers der gegebenen Funktion:")
    function_to_latex(relativ_err)

    return(results)


Beispiel calc_with_err:

In [ ]:
#Beispieleingabe für die Funktion "uncertainty" (Daten aus Jens' Fehlerrechner)
a,b,c,d = sp.symbols('a b c d')
params = a,b,c,d
function = (a*b**2*sp.sqrt(c) - d)
testd = np.array([[0.8,0.3,4.2,0,2.4,0.1,7,0.12], 
                  [6.8,6.3,4.2,6,2.4,6.1,7,6.12]])

z = calc_everything(function, params, testd, params_without_error = [])

## Export der Werte

In [ ]:
def placeholder():
    """
    Erklärung der Methode
    """
    return

### Export als Tabelle

In [99]:
def placeholder():
    """
    Erklärung der Methode
    """
    return